## Feature and model imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../affecteval"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import biosppy as bp
import affecteval
import heartpy as hp
import neurokit2 as nk
import numpy as np
import pandas as pd
import pyhrv
import pyhrv.time_domain as td
import scipy

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier


In [2]:
# Statistical/common feature extraction methods

WINDOW_SIZE = 60
OVERLAP = 30

def extract_mean(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.mean(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.mean(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_med(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.median(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.median(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_std(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.std(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.std(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_var(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.var(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.var(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_range(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.max(segment) - np.min(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.max(segment) - np.min(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_peak(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.max(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.max(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_slope(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        feature = np.gradient(segment)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = np.gradient(segment)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

In [3]:
# Extract power from frequency bands

def bandpower(x, fs, fmin, fmax):
    f, Pxx = scipy.signal.periodogram(x, fs=fs)
    ind_min = np.argmax(f > fmin) - 1
    ind_max = np.argmax(f > fmax) - 1
    return np.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

def extract_freq_power(data, fs, low, high):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        # extract features
        feature = bandpower(segment, fs, low, high)
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = bandpower(segment, fs, low, high)
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

In [4]:
# ECG feature extraction methods

def extract_ecg_features_pyhrv(data, fs):
    n = data.size
    if n == 0:
        print("ECG signal has length 0, returning None")
        return None
    
    hr = []
    rmssd = []
    sdnn = []

    start = 0
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)
    stop = start + window_size
    if stop >= n:
        t, filtered_signal, rpeaks, _, _, _, bpm = bp.signals.ecg.ecg(signal=data, sampling_rate=fs, show=False)
        bpm = np.mean(bpm)
        rmssd_segment = td.rmssd(rpeaks=t[rpeaks])["rmssd"]
        sdnn_segment = td.sdnn(rpeaks=t[rpeaks])["sdnn"]

        hr.append(bpm)
        rmssd.append(rmssd_segment)
        sdnn.append(sdnn_segment)
    else:
        while stop < n:
            stop = start + window_size
            segment = data[start:stop]
            segment, info = nk.ecg_process(segment, sampling_rate=fs)
            segment = segment["ECG_Clean"]
            t, filtered_signal, rpeaks, _, _, _, bpm = bp.signals.ecg.ecg(signal=segment, sampling_rate=fs, show=False)
            try:
                segment = data.iloc[start:stop]
            except AttributeError:
                segment = data[start:stop]
            try:
                bpm = np.mean(bpm)
                rmssd_segment = td.rmssd(rpeaks=t[rpeaks])["rmssd"]
                sdnn_segment = td.sdnn(rpeaks=t[rpeaks])["sdnn"]
            except Exception as e:
                bpm = np.nan
                rmssd_segment = np.nan
                sdnn_segment = np.nan
            hr.append(bpm)
            rmssd.append(rmssd_segment)
            sdnn.append(sdnn_segment)
            start = stop - overlap
    return hr, rmssd, sdnn
    
def extract_hr(data, fs):
    hr, _, _ = extract_ecg_features_pyhrv(data, fs)
    return hr

def extract_rmssd(data, fs):
    _, rmssd, _ = extract_ecg_features_pyhrv(data, fs)
    return rmssd

def extract_sdnn(data, fs):
    _, _, sdnn = extract_ecg_features_pyhrv(data, fs)
    return sdnn

def extract_hr_mean(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        t, filtered_signal, rpeaks, _, _, _, bpm = bp.signals.ecg.ecg(signal=data, sampling_rate=fs, show=False)
        bpm = np.mean(bpm)
        out.append(bpm)
    else:
        while stop < n:
            stop = start + window_size
            segment = data[start:stop]
            segment, info = nk.ecg_process(segment, sampling_rate=fs)
            segment = segment["ECG_Clean"]
            t, filtered_signal, rpeaks, _, _, _, bpm = bp.signals.ecg.ecg(signal=segment, sampling_rate=fs, show=False)
            try:
                segment = data.iloc[start:stop]
            except AttributeError:
                segment = data[start:stop]
            try:
                bpm = np.mean(bpm)
            except Exception as e:
                bpm = np.nan
            out.append(bpm)
            start = stop - overlap
    out = [np.mean(out)]
    return out

def extract_tinn(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        # extract features
        feature = pyhrv.hrv.hrv(
            signal=segment, sampling_rate=fs, plot_ecg=False, plot_Tachogram=False, show=False
        )["tinn"]
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = pyhrv.hrv.hrv(
            signal=segment, sampling_rate=fs, plot_ecg=False, plot_Tachogram=False, show=False
        )["tinn"]
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_nn50(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    n = data.size
    out = []

    if stop >= n:
        segment = data
        # extract features
        feature = pyhrv.hrv.hrv(
            signal=segment, sampling_rate=fs, plot_ecg=False, plot_Tachogram=False, show=False
        )["nn50"]
        out.append(feature)

    while stop < n:
        stop = start + window_size
        segment = data[start:stop]
        # extract features
        feature = pyhrv.hrv.hrv(
            signal=segment, sampling_rate=fs, plot_ecg=False, plot_Tachogram=False, show=False
        )["nn50"]
        out.append(feature)
        start = stop - overlap

    out = list(out)
    return out

def extract_ulf(data, fs):
    low = 0
    high = 0.03
    return extract_freq_power(data, fs, low, high)

def extract_lf(data, fs):
    low = 0.03
    high = 0.5
    return extract_freq_power(data, fs, low, high)

def extract_hf(data, fs):
    low = 0.12
    high = 0.488
    return extract_freq_power(data, fs, low, high)

def extract_uhf(data, fs):
    low = 150
    high = 250
    return extract_freq_power(data, fs, low, high)

def extract_lf_hf_ratio(data, fs):
    lf = extract_lf(data, fs)
    hf = extract_hf(data, fs)
    print(lf)
    print(np.divide(lf, hf))
    return np.divide(lf, hf)

In [ ]:
# EDA feature extraction

# Minimum threshold by which to exclude SCRs (peaks) as relative to the largest amplitude in the signal (from neurokit documentation)
MIN_AMP = 0.3 

def extract_eda_features_nk(signal, fs):
    signal = signal.astype(np.double)
    signal = hp.scale_data(signal)
    signal = scipy.ndimage.median_filter(signal, int(fs))
    signals, info = nk.eda_process(signal, sampling_rate=fs)
    phasic = signals["EDA_Phasic"].to_numpy()
    tonic = signals["EDA_Tonic"].to_numpy()

    peak_info = nk.eda_findpeaks(phasic, fs, amplitude_min=MIN_AMP)
    peak_idx = peak_info["SCR_Peaks"].astype(int)
    peak_amps = peak_info["SCR_Height"]
    peaks = np.zeros(phasic.shape)
    np.put(peaks, peak_idx, [1])
    tonic = tonic - peaks

    return tonic, peaks

def extract_mean_scl(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    out = []

    tonic, _ = extract_eda_features_nk(data, fs)

    if tonic is None:
        print("mean SCL is None")
        return None
    
    n = tonic.size

    if stop >= n:
        segment = tonic
        segment_mean = np.mean(segment)
        out.append(segment_mean)
    while stop < n:
        stop = start + window_size
        segment = tonic[start:stop]
        segment_mean = np.mean(segment)
        out.append(segment_mean)
        start = stop - overlap
    mean_scl = list(out)
    return mean_scl

def extract_scr_rate(data, fs):
    window_size = int(WINDOW_SIZE*fs)
    overlap = int(OVERLAP*fs)

    start = 0
    stop = start + window_size
    out = []

    _, peaks = extract_eda_features_nk(data, fs)

    if peaks is None:
        print("SCR rate is None")
        return None

    n = peaks.size
    
    if stop >= n:
        segment = peaks
        num_peaks = sum(segment)
        out.append(num_peaks)
    while stop < n:
        stop = start + window_size
        segment = peaks[start:stop]
        num_peaks = sum(segment)
        out.append(num_peaks)
        start = stop - overlap
    scr_rate = list(out)
    return scr_rate


In [ ]:
# Set up parameters
from affecteval.signals import Signals, Features


signal_types = [
    Signals.ECG,
    Signals.EDA
]
feature_names = [
    Features.ECG_MEAN, Features.ECG_MEDIAN, Features.ECG_STD, Features.ECG_VAR,
    Features.HR, Features.RMSSD, Features.SDNN,
    Features.ULF, Features.LF, Features.HF, Features.LF_HF,
    Features.EDA_MEAN, Features.EDA_MEDIAN, Features.EDA_STD, Features.EDA_VAR,
    Features.MEAN_SCL, Features.SCR_RATE
]

# Uses default preprocessing methods

feature_extraction_methods = {
    "ECG": {
        Features.ECG_MEAN: extract_mean,
        Features.ECG_MEDIAN: extract_med,
        Features.ECG_STD: extract_std,
        Features.ECG_VAR: extract_var,
        # Features.HR: extract_hr, Features.RMSSD: extract_rmssd, Features.SDNN: extract_sdnn,
        # Features.LF: extract_lf, Features.HF: extract_hf, Features.LF_HF: extract_lf_hf_ratio,
    },
    "EDA": {
        Features.EDA_MEAN: extract_mean,
        Features.EDA_MEDIAN: extract_med,
        Features.EDA_STD: extract_std,
        Features.EDA_VAR: extract_var,
        Features.MEAN_SCL: extract_mean_scl, Features.SCR_RATE: extract_scr_rate
    }
}

## APD

#### NOTE:
The following subjects did not complete the speech exposure phase and were removed:
- 57
- 93
- 16
- 87
- 8
- 21
- 88
- 84
- 23

The following subjects did not complete the bug exposure task and were removed: 
- 4
- 15
- 78

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../affecteval"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import apd

ROOT_DIR = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforME"
DATA_DIR = os.path.join(ROOT_DIR, "data")
APD_PATH = os.path.join(DATA_DIR, "APD")
SOURCE_FOLDER = os.path.join(APD_PATH, "formatted")
# SOURCE_FOLDER = os.path.join(APD_PATH, "test")
METRICS = os.path.join(DATA_DIR, "metrics", "APD")

ALL = "all"
HA = "high_anxiety_group"
LA = "low_anxiety_group"

ha_participant_indices = [
    '4', '6', '7', '8', '10', '12', '15', '16', '18', '22', '26', '27', '29', '31', '32', '33', '35', '42', '45', '47', '48', '49', '54', '55', '66', '69'
]

la_participant_indices = [
    '14', '21', '23', '25', '34', '39', '43', '46', '51', '57', '71', '72', '77', '78', '79', '80', '82', '83', '84', '85', '87', '88', '89', '91', '92', '93'
]

SUBJECTS = ha_participant_indices.extend(la_participant_indices)

In [ ]:
# Rename files to match expected format
def get_data_files(source_folder, signal_types):
        files_dict = {}
        dir_list = [os.path.join(source_folder, f) for f in os.listdir(source_folder)]  # Lists all files and subdirectories
        for p in dir_list:
            if os.path.isdir(p):
                files_p = os.listdir(p)
                s = files_p[0].split("_")[0]  # Get subject index from file
                files_p = [os.path.join(p, f) for f in files_p if any(signal in f for signal in signal_types)]
                files_dict[s] = files_p  # Add list of all files in subdirectory p
            else:
                print(f"Path {p} corresponds to a file, expecting a subdirectory.")
        return files_dict

data_files = get_data_files(SOURCE_FOLDER, ["EDA"])
file = list(data_files.values())[0][0]
df = pd.read_csv(file)
timestamp_col = df.loc[:, "timestamp"]
for subject in data_files.keys():
    files = data_files[subject]
    for file in files:
        df = pd.read_csv(file, index_col=0)
        if df.shape[1] > 2:
            df = df.iloc[:, 2:]
        if "timestamp" not in df.columns:
            df.insert(0, "timestamp", timestamp_col)
        df = df.rename(columns={"Grove sensor reading": "EDA"})
        df.to_csv(file, index=True)


data_files = get_data_files(SOURCE_FOLDER, ["Heart"])
for subject in data_files.keys():
    files = data_files[subject]
    for file in files:
        new_file = file.replace("Heart", "ECG")
        os.rename(file, new_file)

data_files = get_data_files(SOURCE_FOLDER, ["ECG"])
for subject in data_files.keys():
    files = data_files[subject]
    for file in files:
        df = pd.read_csv(file)
        if df.shape[1] > 2:
            df = df.iloc[:, 2:]
        df = df.rename(columns={"ECG reading": "ECG"})
        df.to_csv(file, index=True)

In [50]:
# Generate APD labels
labels = apd.get_suds_labels(APD_PATH)

def generate_labels(data):
    """
    Generate binary labels for APD based on the SUDS questionnaire and the input data format.
    
    Parameters
    --------------------
    :param data: Features to generate labels for. Must include subject ID and phase columns.
    :type data: pd.DataFrame

    Returns
    --------------------
    Generated labels and the unmodified input data.
    """
    annotations = apd.get_suds_labels(APD_PATH)
    labels = []
    for i in range(data.shape[0]):
        subject = int(data["subject"].iloc[i])
        phase = data["Phase"].iloc[i]
        label_row = annotations.loc[(annotations["subject"] == subject)]
        label = label_row[phase]
        labels.append(label)
    labels = np.array(labels).ravel()
    return labels, data

### Run pipeline

In [ ]:
from affecteval.signal_acquisition.signal_acquisition import SignalAcquisition
from affecteval.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from affecteval.feature_extractor.feature_extractor import FeatureExtractor
from affecteval.label_generator.label_generator import LabelGenerator
from affecteval.classification.estimator import Estimator
from affecteval.pipeline.pipeline import Pipeline

import warnings

warnings.filterwarnings("ignore")

label_gen = generate_labels
signal_acq = SignalAcquisition(signal_types=signal_types, source_folder=SOURCE_FOLDER)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=250)
feature_extractor = FeatureExtractor(feature_extraction_methods=feature_extraction_methods, calculate_mean=False)
label_generator = LabelGenerator(label_generation_method=label_gen)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "XGB": XGBClassifier(),
    "RF": RandomForestClassifier()
}

accs = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

aucs = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

true = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

preds = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

estimator_train_val_test = Estimator(2, models, name="Classification: train-val-test", random_seed=36)

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()

# Results
# fitted_models = out[0]
y_true = out[1]
y_preds = out[2]


for model_name in models.keys():
    model = models[model_name]
    acc = accuracy_score(y_true, y_preds[model_name])
    auc = roc_auc_score(y_true, y_preds[model_name])

    true[model_name].append(y_true)
    preds[model_name].append(y_preds[model_name])
    accs[model_name].append(acc)
    aucs[model_name].append(auc)

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...


100%|██████████| 40/40 [00:30<00:00,  1.33it/s]


- Elapsed time: 30.423 s
Running node Feature Extractor...


100%|██████████| 40/40 [02:23<00:00,  3.58s/it]


- Elapsed time: 143.229 s
Running node Label Generator...
- Elapsed time: 0.65 s
Running node Classification: train-val-test...
Cross-validation acc: [0.78651685 0.7943662  0.7943662  0.7943662  0.78873239]
Cross-validation mean acc: 0.7916695679696154
Cross-validation std acc: 0.003376177663247837
Cross-validation f1: [0. 0. 0. 0. 0.]
Cross-validation mean f1: 0.0
Cross-validation std f1: 0.0
Cross-validation auc: [0.60039295 0.67885942 0.54061012 0.58758379 0.55892354]
Cross-validation mean auc: 0.5932739647254366
Cross-validation std auc: 0.04766835378374377
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 292, number of negative: 1128
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2307
[LightGBM] [Info] Number of data points in the train set: 1420, number of used features:

In [77]:
for model_name in models.keys():
    print(f"{model_name} " + "-"*50)
    print(f"Mean accuracy: {np.mean(accs[model_name])}")
    print(f"STD accuracy: {np.std(accs[model_name])}")
    print(f"Mean AUC score: {np.mean(aucs[model_name])}")
    print(f"STD AUC score: {np.std(aucs[model_name])}")
    print()

# Ensemble, average voting:
ensemble_preds = np.mean(list(preds.values()), axis=0)
temp = list(preds.values())
ensemble_preds = [t[0] for t in temp]
ensemble_preds = np.mean(ensemble_preds, axis=0)
ensemble_preds = np.where(ensemble_preds < 0.5, 0, ensemble_preds)
ensemble_preds = np.where(ensemble_preds >= 0.5, 1, ensemble_preds)
ensemble_acc = accuracy_score(list(true.values())[0][0], ensemble_preds)
ensemble_auc = roc_auc_score(list(true.values())[0][0], ensemble_preds)

print("Ensemble " + "-"*50)
print(f"Mean accuracy: {ensemble_acc}")
print(f"Mean AUC score: {ensemble_auc}")

SVM --------------------------------------------------
Mean accuracy: 0.8179775280898877
STD accuracy: 0.0
Mean AUC score: 0.5060975609756098
STD AUC score: 0.0

LGB --------------------------------------------------
Mean accuracy: 0.8741573033707866
STD accuracy: 0.0
Mean AUC score: 0.7057380904387556
STD AUC score: 0.0

XGB --------------------------------------------------
Mean accuracy: 0.8584269662921349
STD accuracy: 0.0
Mean AUC score: 0.6724954646240677
STD AUC score: 0.0

RF --------------------------------------------------
Mean accuracy: 0.8651685393258427
STD accuracy: 0.0
Mean AUC score: 0.6766276960290264
STD AUC score: 0.0

Ensemble --------------------------------------------------
Mean accuracy: 0.8629213483146068
Mean AUC score: 0.6752502855607068


## WESAD

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../affecteval"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import wesad

subject_indices = list(range(2, 12)) + list(range(13, 18))
SUBJECTS = [str(i) for i in subject_indices]

ROOT_DIR = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforME"
DATA_DIR = os.path.join(ROOT_DIR, "data")
WESAD_PATH = os.path.join(DATA_DIR, "WESAD")
SOURCE_FOLDER = os.path.join(WESAD_PATH, "formatted")
ANNOTATIONS_PATH = os.path.join(WESAD_PATH, "annotations")
METRICS = os.path.join(DATA_DIR, "metrics", "WESAD")

In [79]:
# Generate WESAD labels
labels = wesad.generate_labels(ANNOTATIONS_PATH, threshold="dynamic")

def generate_labels(data):
    """
    Generate binary labels for WESAD based on the STAI questionnaire and the input data format.
    
    Parameters
    --------------------
    :param data: Features to generate labels for. Must include subject ID and phase columns.
    :type data: pd.DataFrame

    Returns
    --------------------
    Generated labels and the unmodified input data.
    """
    annotations = wesad.generate_labels(ANNOTATIONS_PATH, threshold="dynamic")
    labels = []
    for i in range(data.shape[0]):
        subject = int(data["subject"].iloc[i])
        phase = data["Phase"].iloc[i]
        label_row = annotations.loc[(annotations["subject"] == subject)]
        label = label_row[phase]
        labels.append(label)
    labels = np.array(labels).ravel()
    return labels, data

### Run pipeline

In [ ]:
from affecteval.signal_acquisition.signal_acquisition import SignalAcquisition
from affecteval.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from affecteval.feature_extractor.feature_extractor import FeatureExtractor
from affecteval.label_generator.label_generator import LabelGenerator
from affecteval.classification.estimator import Estimator
from affecteval.pipeline.pipeline import Pipeline


label_gen = generate_labels
signal_acq = SignalAcquisition(signal_types=signal_types, source_folder=SOURCE_FOLDER)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=250)
feature_extractor = FeatureExtractor(feature_extraction_methods=feature_extraction_methods, calculate_mean=False)
label_generator = LabelGenerator(label_generation_method=label_gen)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "XGB": XGBClassifier(),
    "RF": RandomForestClassifier()
}

wesad_accs = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

wesad_aucs = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

wesad_true = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

wesad_preds = {
    "SVM": [],
    "LGB": [],
    "XGB": [],
    "RF": []
}

estimator_train_val_test = Estimator(2, models, name="Classification: train-val-test", random_seed=36)

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()

# Results
# fitted_models = out[0]
wesad_y_true = out[1]
wesad_y_preds = out[2]


for model_name in models.keys():
    model = models[model_name]
    acc = accuracy_score(wesad_y_true, wesad_y_preds[model_name])
    auc = roc_auc_score(wesad_y_true, wesad_y_preds[model_name])

    wesad_true[model_name].append(wesad_y_true)
    wesad_preds[model_name].append(wesad_y_preds[model_name])
    wesad_accs[model_name].append(acc)
    wesad_aucs[model_name].append(auc)

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...


100%|██████████| 15/15 [00:27<00:00,  1.85s/it]


- Elapsed time: 28.604 s
Running node Feature Extractor...


100%|██████████| 15/15 [01:01<00:00,  4.08s/it]


- Elapsed time: 61.246 s
Running node Label Generator...
- Elapsed time: 0.423 s
Running node Classification: train-val-test...
Cross-validation acc: [0.55172414 0.5        0.5862069  0.59537572 0.62427746]
Cross-validation mean acc: 0.5715168427347019
Cross-validation std acc: 0.04260313642327411
Cross-validation f1: [0.48684211 0.34586466 0.55       0.36363636 0.64480874]
Cross-validation mean f1: 0.47823037474461116
Cross-validation std f1: 0.11280590695767197
Cross-validation auc: [0.59726871 0.58254117 0.56028464 0.70103093 0.63185024]
Cross-validation mean auc: 0.6145951386029676
Cross-validation std auc: 0.049083961871934666
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 305, number of negative: 389
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2100
[LightGBM] [Info]

In [82]:
for model_name in models.keys():
    print(f"{model_name} " + "-"*50)
    print(f"Mean accuracy: {np.mean(wesad_accs[model_name])}")
    print(f"STD accuracy: {np.std(wesad_accs[model_name])}")
    print(f"Mean AUC score: {np.mean(wesad_aucs[model_name])}")
    print(f"STD AUC score: {np.std(wesad_aucs[model_name])}")
    print()

# Ensemble, average voting:
ensemble_preds = np.mean(list(wesad_preds.values()), axis=0)
temp = list(preds.values())
ensemble_preds = [t[0] for t in temp]
ensemble_preds = np.mean(ensemble_preds, axis=0)
ensemble_preds = np.where(ensemble_preds < 0.5, 0, ensemble_preds)
ensemble_preds = np.where(ensemble_preds >= 0.5, 1, ensemble_preds)
ensemble_acc = accuracy_score(list(true.values())[0][0], ensemble_preds)
ensemble_auc = roc_auc_score(list(true.values())[0][0], ensemble_preds)

print("Ensemble " + "-"*50)
print(f"Mean accuracy: {ensemble_acc}")
print(f"Mean AUC score: {ensemble_auc}")

SVM --------------------------------------------------
Mean accuracy: 0.5504587155963303
STD accuracy: 0.0
Mean AUC score: 0.5439209337990969
STD AUC score: 0.0

LGB --------------------------------------------------
Mean accuracy: 0.9862385321100917
STD accuracy: 0.0
Mean AUC score: 0.986580898014825
STD AUC score: 0.0

XGB --------------------------------------------------
Mean accuracy: 0.9770642201834863
STD accuracy: 0.0
Mean AUC score: 0.9772940274346086
STD AUC score: 0.0

RF --------------------------------------------------
Mean accuracy: 0.981651376146789
STD accuracy: 0.0
Mean AUC score: 0.9814262588395672
STD AUC score: 0.0

Ensemble --------------------------------------------------
Mean accuracy: 0.8629213483146068
Mean AUC score: 0.6752502855607068


In [ ]:
ROOT_DIR = "/Users/emilyzhou/Desktop/Research/CAREForMe/"
DATA_DIR = os.path.join(ROOT_DIR, "data")
WESAD_PATH = os.path.join(DATA_DIR, "WESAD")
SOURCE_FOLDER = os.path.join(WESAD_PATH, "original")

file = os.path.join(SOURCE_FOLDER, "S2", "S2.pkl")
data = pd.read_pickle(file)
labels = data["label"]
phase = 1 # baseline
# phase = 2 # stress
# phase = 3 # amusement
indices = [i for i, x in enumerate(labels) if x == phase]
signal = data["signal"]["chest"]["ECG"]
phase = [signal[i] for i in indices]
# print(indices[0:500])
print(len(signal))

4255300
